In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image, PageBreak
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
import tempfile
import os

# === 1. Leer datos desde Excel ===
df_ventas = pd.read_excel("datos_dashboard.xlsx", sheet_name="Ventas")
df_kpis = pd.read_excel("datos_dashboard.xlsx", sheet_name="KPIs")

# === 2. Crear gráficas ===
# Línea por producto
plt.figure(figsize=(6, 3))
for i, row in df_ventas.iterrows():
    plt.plot(["Enero", "Febrero", "Marzo"], row[1:4], label=row["Producto"])
plt.title("Ventas por producto (Enero-Marzo)")
plt.legend()
plt.tight_layout()
img_line = os.path.join(tempfile.gettempdir(), "grafica_lineas.png")
plt.savefig(img_line)
plt.close()

# Barras por total
plt.figure(figsize=(6, 3))
plt.bar(df_ventas["Producto"], df_ventas["Total Q1"], color='skyblue')
plt.title("Total Ventas por Producto (Q1)")
plt.tight_layout()
img_bar = os.path.join(tempfile.gettempdir(), "grafica_barras.png")
plt.savefig(img_bar)
plt.close()

# === 3. Crear PDF tipo dashboard ===
output_path = "dashboard_desde_excel.pdf"
doc = SimpleDocTemplate(output_path, pagesize=A4)
styles = getSampleStyleSheet()
elements = []

# Título
elements.append(Paragraph("📊 Dashboard Ejecutivo - Q1", styles['Heading1']))
elements.append(Spacer(1, 12))

# KPIs
elements.append(Paragraph("Indicadores Clave", styles['Heading2']))
elements.append(Spacer(1, 6))
tabla_kpis = Table([df_kpis.columns.tolist()] + df_kpis.values.tolist(), colWidths=[200, 150])
tabla_kpis.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#003366")),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
    ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTSIZE', (0, 0), (-1, -1), 10),
    ('BACKGROUND', (0, 1), (-1, -1), colors.whitesmoke),
]))
elements.append(tabla_kpis)
elements.append(Spacer(1, 20))

# Gráficas
elements.append(Paragraph("Tendencia de Ventas Mensuales", styles['Heading2']))
elements.append(Image(img_line, width=400, height=200))
elements.append(Spacer(1, 12))

elements.append(Paragraph("Ventas Totales por Producto", styles['Heading2']))
elements.append(Image(img_bar, width=400, height=200))
elements.append(PageBreak())

# Tabla de ventas
elements.append(Paragraph("Detalle de Ventas por Producto", styles['Heading2']))
tabla_ventas = Table([df_ventas.columns.tolist()] + df_ventas.values.tolist())
tabla_ventas.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#003366")),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
    ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTSIZE', (0, 0), (-1, -1), 9),
    ('BACKGROUND', (0, 1), (-1, -1), colors.whitesmoke),
]))
elements.append(tabla_ventas)

# Crear el PDF
doc.build(elements)

print(f"Dashboard generado: {output_path}")



Dashboard generado: dashboard_desde_excel.pdf
